In [1]:
%matplotlib widget
import time
import rospy
import select
import actionlib
import actionlib.msg
import assignment_2_2023.msg
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.animation import FuncAnimation
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from assignment_2_2023.msg import Info, PlanningActionFeedback
from geometry_msgs.msg import Point, Pose, Twist

# Initialize the service node
rospy.init_node('action_client')
# Creating a ROS publisher to publish on /robot_pos_vel topic the position and velocity of Robot
pub = rospy.Publisher('/robot_pos_vel', Info, queue_size=10)
# Execution of client request to the server
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
# Block the execution until communication with server is established
client.wait_for_server()

True

In [2]:
x_target = []
y_target = []
x_canceled = []
y_canceled = []
x_reached = []
y_reached = []

def set_goal(b):
    global sliders_displayed
    
    if not sliders_displayed:
        clear_output(wait=True)
        display(hbox)
        sliders_displayed = True
        display(x_slider)
        display(y_slider)
        display(send_button)
        
    
def cancel_goal(b):
    global sliders_displayed, goal_canceled, x_target, y_target, x_canceled, y_canceled
    
    if not goal_canceled:
        goal_canceled = True
        clear_output(wait=True)
        display(hbox)
        # Cancel the goal
        client.cancel_goal()
        x_target.remove(goal.target_pose.pose.position.x)
        y_target.remove(goal.target_pose.pose.position.y)
        x_canceled.append(goal.target_pose.pose.position.x)
        y_canceled.append(goal.target_pose.pose.position.y)
        # Print the goal status
        goal_status.value = f"Goal canceled! x:{goal.target_pose.pose.position.x}, y:{goal.target_pose.pose.position.y}"
        display(goal_status)
    
    
def send_goal(b):
    global sliders_displayed, goal_canceled, goal, x_target, y_target
    
    clear_output(wait=True)
    display(hbox)
    sliders_displayed = False
    goal_canceled = False
    # Initialize an instance of PlanningGoal() to pass the goal coordinates.
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x_slider.value
    goal.target_pose.pose.position.y = y_slider.value
    # Save the goals coordinates
    x_target.append(goal.target_pose.pose.position.x)
    y_target.append(goal.target_pose.pose.position.y)
    # Send the goal
    client.send_goal(goal, None, None, vis.clbk_feedback)
    # Print the goal status
    goal_status.value = f"Goal set! x:{goal.target_pose.pose.position.x}, y:{goal.target_pose.pose.position.y}"
    display(goal_status)

In [3]:
# Define widgets
set_button = widgets.Button(description='Set Goal', button_style='success')
cancel_button = widgets.Button(description='Cancel Goal', button_style='danger')
send_button = widgets.Button(description='Send Coordinate', button_style='info')
goal_status = widgets.Textarea(layout=widgets.Layout(width='auto'))

# Attach the callback function to each button
set_button.on_click(set_goal)
cancel_button.on_click(cancel_goal)
send_button.on_click(send_goal)

# Create slider widgets for x and y values
x_slider = widgets.FloatSlider(min=-8, max=8, description='X:')
y_slider = widgets.FloatSlider(min=-8, max=8, description='Y:')

# Variable to track if sliders have been displayed
sliders_displayed = False

# Display the buttons in an HBox
hbox = widgets.HBox([set_button, cancel_button])


In [4]:
%matplotlib notebook
class Visualiser:
    def __init__(self):
        self.fig_hist, self.ax_hist = plt.subplots()
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro', linestyle='None', markersize=5, markeredgecolor='black', label='Robot')
        self.target, = plt.plot([], [], marker='o', markersize=10, markeredgecolor='black', linestyle='None', color='blue', label='Target')
        self.reached, = plt.plot([], [], marker='o', markersize=10, markeredgecolor='black', linestyle='None', color='green', label='Target reached')
        self.canceled, = plt.plot([], [], marker='x', markersize=10, linestyle='None', color='red', label='Target canceled')
        self.x_data, self.y_data = [] , []

    def plot_init(self):
        self.ax.set_xlim(10, -10)
        self.ax.set_ylim(10, -10)
        self.ax.grid(True)
        self.ax.legend()
        return self.ax,

    def hist_init(self):
        self.ax_hist.set_title("Goals Status")
        self.ax_hist.set_xlabel("Status")
        self.ax_hist.set_ylabel("Count")
        self.ax_hist.set_ylim(0, 10)  # Set an initial limit for the y-axis
        return self.ax_hist,
    
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        # Initialize a new message
        # The struct of Info() is (x,y,vel_x,vel_z)
        new_info = Info()

        # Retrieve the position and velocity from geometry_msgs
        # and save the values inside the new message
        new_info.x = msg.pose.pose.position.x           # x position coordinate
        new_info.y = msg.pose.pose.position.y           # y position coordinate
        new_info.vel_x = msg.twist.twist.linear.x       # linear velocity along x axis
        new_info.vel_z = msg.twist.twist.angular.z      # angular velocity around z axis

        # Pubblish new message on /robot_pos_vel topic 
        pub.publish(new_info)
        
    def clbk_feedback(self, feedback):
        # Callback function that process the feedback from client.
        # :param feedback: feedback from the target as "Target reached!" or "Target canceled!".
        global goal_canceled, x_canceled, y_canceled
        if feedback.stat == "Target reached!":
            # To avoid cancel the goal once the robot has reached it
            goal_canceled = True
            x_reached.append(goal.target_pose.pose.position.x)
            y_reached.append(goal.target_pose.pose.position.y)
            x_target.remove(goal.target_pose.pose.position.x)
            y_target.remove(goal.target_pose.pose.position.y)
            # Print the goal status
            goal_status.value = f"Goal reached! x:{goal.target_pose.pose.position.x}, y:{goal.target_pose.pose.position.y}"
            display(goal_status)

    def update_plot(self, frame):
        global x_target, y_target, x_canceled, y_canceled, x_reached, y_reached
        self.ln.set_data(self.x_data, self.y_data)
        self.target.set_data(x_target, y_target)
        self.canceled.set_data(x_canceled, y_canceled)
        self.reached.set_data(x_reached, y_reached)

    def update_hist(self, frame):
        global x_reached
        self.ax_hist.clear()  # Clear the histogram axes
        self.hist_init()  # Re-initialize the histogram plot settings
        data = [len(x_reached), len(x_canceled)]
        labels = ['Reached', 'Canceled']
        self.ax_hist.bar(labels, data, color=['green', 'red'], alpha=0.7, edgecolor='black')
        
vis = Visualiser()
# Ros subscriber to get the robot's odometry
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
# Plot the environment
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
# Plot the histogram
ani_hist = FuncAnimation(vis.fig_hist, vis.update_hist, init_func=vis.hist_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
display(hbox)

Textarea(value='Goal set! x:5.5, y:5.8', layout=Layout(width='auto'))